##### This notebook is an assignment in udacity data engineering project with Azure.
##### In this project, it requires:
- Design a star schema based on the business outcomes below <br>
- Import the data into Azure Databricks using Delta Lake to create a Bronze data store <br>
- Create a gold data store in Delta Lake tables <br>
- Transform the data into the star schema for a Gold data store <br>

##### Steps in this notebook:
- Extract data from csv to DBFS
- Refine tables add column names to each table
- Write data into delta file system
- Create delta tables
- Transform data to star schema
- Write star schema into delta tables

In [0]:
from pyspark.sql import functions as f
from pyspark.sql.functions import sequence, explode, to_date

##### Extract data from csv to DBFS

In [0]:
# ingest data from DBFS and create dataframe
# payments
df_payments = spark.read.format("csv") \
            .option("inferSchema", "false") \
            .option("header", "false") \
            .option("sep", ",") \
            .load("/FileStore/tables/payments/payments.csv")

# trips
df_trips = spark.read.format("csv") \
            .option("inferSchema", "false") \
            .option("header", "false") \
            .option("sep", ",") \
            .load("/FileStore/tables/trips/trips.csv")

# riders
df_riders = spark.read.format("csv") \
            .option("inferSchema", "false") \
            .option("header", "false") \
            .option("sep", ",") \
            .load("/FileStore/tables/riders/riders.csv")

# stations
df_stations = spark.read.format("csv") \
            .option("inferSchema", "false") \
            .option("header", "false") \
            .option("sep", ",") \
            .load("/FileStore/tables/stations/stations.csv")

In [0]:
# validate 4 dataframes
df_payments.take(5)

Out[3]: [Row(_c0='1', _c1='2019-05-01', _c2='9.0', _c3='1000'),
 Row(_c0='2', _c1='2019-06-01', _c2='9.0', _c3='1000'),
 Row(_c0='3', _c1='2019-07-01', _c2='9.0', _c3='1000'),
 Row(_c0='4', _c1='2019-08-01', _c2='9.0', _c3='1000'),
 Row(_c0='5', _c1='2019-09-01', _c2='9.0', _c3='1000')]

In [0]:
df_riders.take(5)

Out[4]: [Row(_c0='1000', _c1='Diana', _c2='Clark', _c3='1200 Alyssa Squares', _c4='1989-02-13', _c5='2019-04-23', _c6=None, _c7='True'),
 Row(_c0='1001', _c1='Jennifer', _c2='Smith', _c3='397 Diana Ferry', _c4='1976-08-10', _c5='2019-11-01', _c6='2020-09-01', _c7='True'),
 Row(_c0='1002', _c1='Karen', _c2='Smith', _c3='644 Brittany Row Apt. 097', _c4='1998-08-10', _c5='2022-02-04', _c6=None, _c7='True'),
 Row(_c0='1003', _c1='Bryan', _c2='Roberts', _c3='996 Dickerson Turnpike', _c4='1999-03-29', _c5='2019-08-26', _c6=None, _c7='False'),
 Row(_c0='1004', _c1='Jesse', _c2='Middleton', _c3='7009 Nathan Expressway', _c4='1969-04-11', _c5='2019-09-14', _c6=None, _c7='True')]

In [0]:
df_stations.take(5)

Out[5]: [Row(_c0='525', _c1='Glenwood Ave & Touhy Ave', _c2='42.012701', _c3='-87.66605799999999'),
 Row(_c0='KA1503000012', _c1='Clark St & Lake St', _c2='41.88579466666667', _c3='-87.63110066666668'),
 Row(_c0='637', _c1='Wood St & Chicago Ave', _c2='41.895634', _c3='-87.672069'),
 Row(_c0='13216', _c1='State St & 33rd St', _c2='41.8347335', _c3='-87.6258275'),
 Row(_c0='18003', _c1='Fairbanks St & Superior St', _c2='41.89580766666667', _c3='-87.62025316666669')]

In [0]:
df_trips.take(5)

Out[6]: [Row(_c0='89E7AA6C29227EFF', _c1='classic_bike', _c2='2021-02-12 16:14:56', _c3='2021-02-12 16:21:43', _c4='525', _c5='660', _c6='71934'),
 Row(_c0='0FEFDE2603568365', _c1='classic_bike', _c2='2021-02-14 17:52:38', _c3='2021-02-14 18:12:09', _c4='525', _c5='16806', _c6='47854'),
 Row(_c0='E6159D746B2DBB91', _c1='electric_bike', _c2='2021-02-09 19:10:18', _c3='2021-02-09 19:19:10', _c4='KA1503000012', _c5='TA1305000029', _c6='70870'),
 Row(_c0='B32D3199F1C2E75B', _c1='classic_bike', _c2='2021-02-02 17:49:41', _c3='2021-02-02 17:54:06', _c4='637', _c5='TA1305000034', _c6='58974'),
 Row(_c0='83E463F23575F4BF', _c1='electric_bike', _c2='2021-02-23 15:07:23', _c3='2021-02-23 15:22:37', _c4='13216', _c5='TA1309000055', _c6='39608')]

##### Refine tables add column names to each table

In [0]:
# add column names to each dataframe
# payments
df_payments = df_payments.withColumnRenamed("_c0", "payment_id") \
        .withColumnRenamed("_c1", "date") \
        .withColumnRenamed("_c2", "amount") \
        .withColumnRenamed("_c3", "rider_id")

In [0]:
# add column names to each dataframe
# riders
df_riders = df_riders.withColumnRenamed("_c0", "rider_id") \
        .withColumnRenamed("_c1", "first") \
        .withColumnRenamed("_c2", "last") \
        .withColumnRenamed("_c3", "address") \
        .withColumnRenamed("_c4", "birthday") \
        .withColumnRenamed("_c5", "startdate") \
        .withColumnRenamed("_c6", "enddate") \
        .withColumnRenamed("_c7", "member")

In [0]:
# add column names to each dataframe
# trips
df_trips = df_trips.withColumnRenamed("_c0", "trip_id") \
        .withColumnRenamed("_c1", "rideable_type") \
        .withColumnRenamed("_c2", "started_at") \
        .withColumnRenamed("_c3", "ended_at") \
        .withColumnRenamed("_c4", "start_station_id") \
        .withColumnRenamed("_c5", "end_station_id") \
        .withColumnRenamed("_c6", "rider_id")

In [0]:
# add column names to each dataframe
# stations
df_stations= df_stations.withColumnRenamed("_c0", "station_id") \
        .withColumnRenamed("_c1", "name") \
        .withColumnRenamed("_c2", "latitude") \
        .withColumnRenamed("_c3", "longitude")

In [0]:
df_payments.take(2)

Out[14]: [Row(payment_id='1', date='2019-05-01', amount='9.0', rider_id='1000'),
 Row(payment_id='2', date='2019-06-01', amount='9.0', rider_id='1000')]

##### Write data into delta file system

In [0]:
# write data into delta lake
df_payments.write.format("delta") \
    .mode("overwrite") \
    .save("/delta/payments")

df_trips.write.format("delta") \
    .mode("overwrite") \
    .save("/delta/trips")

df_stations.write.format("delta") \
    .mode("overwrite") \
    .save("/delta/stations")

df_riders.write.format("delta") \
    .mode("overwrite") \
    .save("/delta/riders")

##### Create delta tables

In [0]:
# load data
'''
df_payments = spark.read.format("delta") \
    .load("/delta/payments")

# write into tables
df_payments.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("payments")
'''

In [0]:
# load data
'''
df_riders = spark.read.format("delta") \
    .load("/delta/riders")

# write into tables
df_riders.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("riders")

# load data
df_trips = spark.read.format("delta") \
    .load("/delta/trips")

# write into tables
df_trips.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("trips")

# load data
df_stations = spark.read.format("delta") \
    .load("/delta/stations")

# write into tables
df_stations.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("stations")
'''

In [0]:
# write into tables
spark.sql("CREATE TABLE payments USING DELTA LOCATION '/delta/payments'")
spark.sql("CREATE TABLE trips USING DELTA LOCATION '/delta/trips'")
spark.sql("CREATE TABLE riders USING DELTA LOCATION '/delta/riders'")
spark.sql("CREATE TABLE stations USING DELTA LOCATION '/delta/stations'")

Out[4]: DataFrame[]

##### Transform data to star schema

In [0]:
# read all tables from delta
df_payments = spark.read.format("delta") \
    .load("/delta/payments")

df_riders = spark.read.format("delta") \
    .load("/delta/riders")

df_trips = spark.read.format("delta") \
    .load("/delta/trips")

df_stations = spark.read.format("delta") \
    .load("/delta/stations")

In [0]:
# process df_trips into star schema

def process_trips(in_df):
    ''' 
    this function process df_trips to dim_trips in star schema
    includes creating additional columns and adjusting data formats
    args: dataframe
    returns: dataframe
    
    '''
    # select needed cols
    out_df = in_df.select('trip_id', 'rider_id', 'rideable_type', 'start_station_id', 'end_station_id', 'started_at', 'ended_at')
    
    # convert to correct data format
    out_df = out_df.withColumn('rider_id', f.col('rider_id').cast('int')) \
        .withColumn('started_at', f.to_timestamp('started_at')) \
        .withColumn('ended_at', f.to_timestamp('ended_at')) \
    
    # create duration column
    out_df = out_df.withColumn('duration', f.col('ended_at').cast('long') - f.col('started_at').cast('long'))
    
    # create rider age column
    merged_df = out_df.join(df_riders.select('birthday','rider_id'), on='rider_id', how='left')
    out_df = merged_df.withColumn('rider_age', f.datediff(f.col('ended_at'), f.col('birthday'))/360) \
        .withColumn('rider_age', f.round(f.col('rider_age'), 0))
    
    # drop added birthday
    out_df = out_df.drop('birthday')
    
    return out_df

In [0]:
fact_trips = process_trips(df_trips)

In [0]:
# process payments
def process_payments(in_df):
    ''' 
    this function process df_payments to fact_payments in star schema
    includes adjusting data formats
    args: dataframe
    returns: dataframe
    
    '''
    # select needed cols
    out_df = in_df.select('payment_id', 'date', 'amount', 'rider_id')
    
    # convert to correct data format
    out_df = out_df.withColumn('payment_id', f.col('payment_id').cast('int')) \
    .withColumn('rider_id', f.col('rider_id').cast('int')) \
    .withColumn('amount', f.col('amount').cast('float')) \
    .withColumn('date', f.col('date').cast('date'))
    
    return out_df

In [0]:
fact_payments = process_payments(df_payments)

In [0]:
# process riders
def process_riders(in_df):
    ''' 
    this function process df_riders to dim_riders in star schema
    includes adjusting data formats
    args: dataframe
    returns: dataframe
    
    '''
    # select needed cols
    out_df = in_df.select('rider_id', 'first', 'last', 'address', 'birthday', 'startdate', 'enddate', 'member')
    
    # convert to correct data format
    out_df = out_df.withColumn('rider_id', f.col('rider_id').cast('int')) \
        .withColumn('birthday', f.col('birthday').cast('date')) \
        .withColumn('startdate', f.col('startdate').cast('date')) \
        .withColumn('enddate', f.col('enddate').cast('date')) \
        .withColumn('member', f.col('member').cast('boolean'))
    return out_df

In [0]:
dim_riders = process_riders(df_riders)

In [0]:
# process stations
def process_stations(in_df):
    ''' 
    this function process df_stations to dim_stations in star schema
    includes adjusting data formats
    args: dataframe
    returns: dataframe
    
    '''
    # select needed cols
    out_df = in_df.select('station_id', 'name', 'latitude', 'longitude')
    
    # set nullable to false for primary key
    out_df.schema['station_id'].nullable = False
    # convert to correct data format
    out_df = out_df.withColumn('latitude', f.col('latitude').cast('float')) \
        .withColumn('longitude', f.col('longitude').cast('float'))
    
    return out_df

In [0]:
dim_stations = process_stations(df_stations)

In [0]:
# generate dim_calendar table
# https://stackoverflow.com/questions/43141671/sparksql-on-pyspark-how-to-generate-time-series

def create_calendar():
    ''' 
    this function creates a dim_calendar used as dimension table in star schema
    args: none
    returns: dataframe
    
    '''
    # create date series
    out_df = spark.sql("SELECT sequence(to_date('2010-01-01'), to_date('2022-09-25'), interval 1 day) as date").withColumn("date", explode(f.col("date")))
    # create tother columns
    out_df = out_df.withColumn('quarter', f.quarter(f.col('date'))) \
        .withColumn('year', f.year(f.col('date'))) \
        .withColumn('month', f.month(f.col('date')))
    return out_df

In [0]:
dim_calendar = create_calendar()

##### Write star schema into delta tables

In [0]:
# write star schema tables to delta
dim_stations.write.format('delta').mode('overwrite').saveAsTable('dim_stations')
dim_calendar.write.format('delta').mode('overwrite').saveAsTable('dim_calendar')
dim_riders.write.format('delta').mode('overwrite').saveAsTable('dim_riders')
fact_trips.write.format('delta').mode('overwrite').saveAsTable('fact_trips')
fact_payments.write.format('delta').mode('overwrite').saveAsTable('fact_payments')